In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas

base_url="https://ageofempires.fandom.com"
req_headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}

req=requests.get(base_url + "/wiki/Civilizations_(Age_of_Empires_II)", headers=req_headers)
content=req.content
soup=bs(content,"html.parser")
civs_page=soup.find_all("span", {"class":"mw-headline"})

In [2]:
civilizations=[]
excludables = ["DarkageDE.png","FeudalageDE.png","CastleageDE.png","ImperialageDE.png",
               "Darkage.png","Feudalage.png","Castleage.png","Imperialage.png",
               "Militiaavailable.png",
               "Manatarmsavailable.png","Longswordsmanavailable.png","Spearmanavailable.png",
               "Archeravailable.png","Skirmisheravailable.png",
               "Siegetoweravailable.png","Batteringramavailable.png","Scorpionavailable.png","Mangonelavailable.png",
               "galleyavailable.png","Wargalleyavailable.png","Transportshipavailable.png","Tradecogavailable.png",
               "Fishtrapavailable.png","Fishingshipavailable.png","Careeningavailable.png","Gillnetsavailable.png",
               "Outpostavailable.png","Watchtoweravailable.png","Palisadewallavailable.png","Palisadegateavailable.png","Gateavailable.png",
               "Monknativeavailable.png","Monkavailable.png",
               "Paddedarcherarmoravailable.png","Fletchingavailable.png","Bodkinarrowavailable.png",
               "Forgingavailable.png","Ironcastingavailable.png",
               "Scalemailarmoravailable.png",
               "Chemistryavailable.png","Bombardtowerresearchavailable.png","Ballisticsavailable.png",
               "Guardtowerresearchavailable.png","Keepresearchavailable.png","Murderholesavailable.png",
               "Towncenteravailable.png","Villageravailable.png","Townwatchavailable.png","Townpatrolavailable.png",
               "Feudalageavailable.png","Castleageavailable.png","Imperialageavailable.png","Loomavailable.png","Wheelbarrowavailable.png","Handcartavailable.png",
               "Goldminingavailable.png","Stoneminingavailable.png",
               "Doublebitaxeavailable.png","Bowsawavailable.png",
               "Horsecollaravailable.png","Heavyplowavailable.png",
               "Tradecartavailable.png","Tradecartnativeavailable.png","Coinageavailable.png","Bankingavailable.png","Caravanavailable.png",
               "Houseavailable.png"]

excludablebuildings = ["Barracksavailable.png","Barracks-logo.jpg","Archeryrangeavailable.png","Stableavailable.png",
                       "Siegeworkshopavailable.png","Castleavailable.png","Blacksmithicon.png","Dockavailable.png","Blacksmithavailable.png",
                       "Monasteryavailable.png","Universityavailable.png","Towncenteravailable.png","Miningcampavailable.png",
                       "Lumbercampavailable.png","Millavailable.png","Farmavailable.png","Millicon.png","Marketavailable.png",
                       "Wonderavailable.png"]
for civ in civs_page:
    civilization={}
    civ_url=civ.find("a", href=True)
    if civ_url is None:
        continue
    req_civ=requests.get(base_url + civ_url['href'] + "/Tree", headers=req_headers)
    civ_soup=bs(req_civ.content, "html.parser")
    
    if civ_soup.find("div", {"class": "noarticletext mw-content-ltr"}):
        continue
        
    #get civilization speciality
    civilization['civilization']=civ_url.text
    civilization['specialization']=civ_soup.find("p").text.split(" as ")[1][:-1]
    
    civ_info=civ_soup.find("td")
    
    #get civilization bonuses
    civilization['civilization bonuses']=""
    for bonuses in civ_info.find_all("ul")[6].find_all("li"):
        civilization['civilization bonuses']+=bonuses.text + "\n"
    
    #get civilization unique units
    civilization['unique units']=""
    for units in civ_info.find_all("ul")[7].find_all("li"):
        civilization['unique units']+=units.text + "\n"
         
    #get civilization unique technologies
    civilization['unique technologies']=""
    for technologies in civ_info.find_all("ul")[8].find_all("li"):
        civilization['unique technologies']+=technologies.text + "\n"
    #----------------------------------------------------------------------------------------------------------------
    
    #set default values for the current civ's Tech Tree
    
    #Barrack's Units:
    civilization['militiaLine'] = 1         # all civs can at least reach up to Long Swordsman
    civilization['spearmanLine'] = 1
    civilization['EaglescoutLine'] = 0
    
    
    #Barrack's Techs:
    civilization['has_supplies'] = 0
    civilization['has_squires'] = 0
    civilization['has_arson'] = 0

    #Archeryrange's Units:
    civilization['archerLine'] = 1
    civilization['skirmisherLine'] = 1
    civilization['handcanoneerLine'] = 0
    civilization['cavalryarcherLine'] = 0
    
    #Archeryrange's Techs:
    civilization['has_thumbring'] = 0
    civilization['has_parthiantactics'] = 0
    
    #Stable's Units:
    civilization['scoutLine'] = 0
    civilization['knightLine'] = 0
    civilization['camelriderLine'] = 0
    civilization['battleelephantLine'] = 0
    civilization['steppelancerLine'] = 0
    
    #Stable's Techs:
    civilization['has_bloodlines'] = 0
    civilization['has_husbandry'] = 0
    
    #Siege Workshop's Units:
    civilization['batteringramLine'] = 1
    civilization['mangonelLine'] = 1        # all civs can at least reach up to Onager
    civilization['scorpionLine'] = 1
    civilization['bombardcannonLine'] = 0
        
    #Dock's Units:
    civilization['fireshipLine'] = 0
    civilization['demolitionraftLine'] = 0
    civilization['galleyLine'] = 1          # all civs can at least reach up to War Galley
    civilization['cannongalleonLine'] = 0
    
    #Dock's Techs:
    civilization['has_drydock'] = 0
    civilization['has_shipwright'] = 0
    
    #Defensive Structures:
    civilization['watchtowerLine'] = 1
    civilization['stonewallLine'] = 0
    
    #Blacksmith's Techs:
    civilization['piercearmorLine'] = 1
    civilization['pierceatkLine'] = 1       # all civs can at least reach up to Bodkin Arrow
    civilization['meleeatkLine'] = 1        # all civs can at least reach up to Iron Casting
    civilization['cavalryarmorLine'] = 0
    civilization['infantryarmorLine'] = 1
    
    #Monastery's Techs:
    civilization['has_faith'] = 0
    civilization['has_redemption'] = 0
    civilization['has_illumination'] = 0
    civilization['has_atonement'] = 0
    civilization['has_blockprinting'] = 0
    civilization['has_herbalmedicine'] = 0
    civilization['has_theocracy'] = 0
    civilization['has_heresy'] = 0
    civilization['has_sanctity'] = 0
    civilization['has_fervor'] = 0
    
    #University's Techs:
    civilization['has_masonry'] = 0
    civilization['has_architecture'] = 0
    civilization['has_fortifiedwall'] = 0
    civilization['has_bombardtower'] = 0
    civilization['has_siegeengineers'] = 0
    civilization['has_heatedshot'] = 0
    civilization['has_arrowslits'] = 0
    civilization['has_treadmillcrane'] = 0
    
    #Economy's Techs:
    civilization['goldLine'] = 1
    civilization['stoneLine'] = 1
    civilization['woodLine'] = 1            # all civs can at least reach up to Bow Saw
    civilization['foodLine'] = 1            # all civs can at least reach up to Heavy Plow

    #Market's Techs:
    civilization['has_guilds'] = 0
    
    images = civ_soup.find_all("img", {"class": "lazyload"})
    for item in images:
        if item["data-image-name"] in excludables:
            continue
        if item["data-image-name"] in excludablebuildings:
            continue
            
            
        
        if(item["data-image-name"] == "Twohandedswordsmanavailable.png" or item["data-image-name"] == "Championavailable.png"):
            civilization['militiaLine']+=1
            
        elif(item["data-image-name"] == "Pikemanavailable.png" or item["data-image-name"] == "Halberdieravailable.png"):
            civilization['spearmanLine']+=1
            
        elif(item["data-image-name"] == "Eaglescoutavailable.png" or (item["data-image-name"] == "Eaglewarrioravailable.png" or item["data-image-name"] == "Eliteeaglewarrioravailable.png")):
            civilization['EaglescoutLine']+=1
            
        elif(item["data-image-name"] == "Suppliesavailable.png"):
            civilization['has_supplies']+=1
            
        elif(item["data-image-name"] == "Squiresavailable.png"):
            civilization['has_squires']+=1
            
        elif(item["data-image-name"] == "Arsonavailable.png"):
            civilization['has_arson']+=1
            
        elif(item["data-image-name"] == "Crossbowmanavailable.png" or item["data-image-name"] == "Arbalesteravailable.png"):
            civilization['archerLine']+=1
            
        elif(item["data-image-name"] == "Eliteskirmisheravailable.png"):
            civilization['skirmisherLine']+=1
            
        elif(item["data-image-name"] == "Handcannoneeravailable.png"):
            civilization['handcanoneerLine']+=1
            
        elif(item["data-image-name"] == "Cavalryarcheravailable.png" or item["data-image-name"] == "Heavycavalryarcheravailable.png"):
            civilization['cavalryarcherLine']+=1
            
        elif(item["data-image-name"] == "Thumbringavailable.png"):
            civilization['has_thumbring']+=1
            
        elif(item["data-image-name"] == "Parthiantacticsavailable.png"):
            civilization['has_parthiantactics']+=1
            
        elif(item["data-image-name"] == "Scoutcavalryavailable.png" or (item["data-image-name"] == "Lightcavalryavailable.png" or item["data-image-name"] == "Hussaravailable.png")):
            civilization['scoutLine']+=1
            
        elif(item["data-image-name"] == "Knightavailable.png" or (item["data-image-name"] =="Cavalieravailable.png" or item["data-image-name"] == "Paladinavailable.png")):
            civilization['knightLine']+=1
            
        elif(item["data-image-name"] == "Camelrideravailable.png" or item["data-image-name"] == "Heavycamelrideravailable.png"):
            civilization['camelriderLine']+=1
            
        elif(item["data-image-name"] == "Battleelephantavailable.png" or item["data-image-name"] == "Elitebattleelephantavailable.png"):
            civilization['battleelephantLine']+=1
            
        elif(item["data-image-name"] == "Steppelanceravailable.png" or item["data-image-name"] == "Elitesteppelanceravailable.png"):
            civilization['steppelancerLine']+=1
            
        elif(item["data-image-name"] == "Bloodlinesavailable.png"):
            civilization['has_bloodlines']+=1
            
        elif(item["data-image-name"] == "Husbandryavailable.png"):
            civilization['has_husbandry']+=1
            
        elif(item["data-image-name"] =="Cappedramavailable.png" or item["data-image-name"] == "Siegeramavailable.png"):
            civilization['batteringramLine']+=1
            
        elif(item["data-image-name"] == "Onageravailable.png" or item["data-image-name"] == "Siegeonageravailable.png"):
            civilization['mangonelLine']+=1
            
        elif(item["data-image-name"] == "Heavyscorpionavailable.png"):
            civilization['scorpionLine']+=1
            
        elif(item["data-image-name"] == "Bombardcannonavailable.png"):
            civilization['bombardcannonLine']+=1
            
        elif(item["data-image-name"] == "Firegalleyavailable.png" or (item["data-image-name"] =="Fireshipavailable.png" or item["data-image-name"] == "Fastfireshipavailable.png")):
            civilization['fireshipLine']+=1
            
        elif(item["data-image-name"] == "Cannongalleonavailable.png" or item["data-image-name"] == "Elitecannongalleonavailable.png"):
            civilization['cannongalleonLine']+=1
            
        elif(item["data-image-name"] == "Demolitionraftavailable.png" or (item["data-image-name"] =="Demolitionshipavailable.png" or item["data-image-name"] == "Heavydemolitionshipavailable.png")):
            civilization['demolitionraftLine']+=1
            
        elif(item["data-image-name"] == "Galleyavailable.png" or item["data-image-name"] == "Galleonavailable.png"):
            civilization['galleyLine']+=1
            
        elif(item["data-image-name"] == "Drydockavailable.png"):
            civilization['has_drydock']+=1
            
        elif(item["data-image-name"] == "Shipwrightavailable.png"):
            civilization['has_shipwright']+=1
            
        elif(item["data-image-name"] == "Guardtoweravailable.png" or item["data-image-name"] == "Keepavailable.png"):
            civilization['watchtowerLine']+=1
            
        elif(item["data-image-name"] == "Stonewallavailable.png" or item["data-image-name"] == "Fortifiedwallavailable.png"):
            civilization['stonewallLine']+=1
            
        elif(item["data-image-name"] == "Leatherarcherarmoravailable.png" or item["data-image-name"] == "Ringarcherarmoravailable.png"):
            civilization['piercearmorLine']+=1
            
        elif(item["data-image-name"] == "Braceravailable.png"):
            civilization['pierceatkLine']+=1
            
        elif(item["data-image-name"] == "Blastfurnaceavailable.png"):
            civilization['meleeatkLine']+=1
            
        elif(item["data-image-name"] == "Scalebardingarmoravailable.png" or (item["data-image-name"] =="Chainbardingarmoravailable.png" or item["data-image-name"] == "Platebardingarmoravailable.png")):
            civilization['cavalryarmorLine']+=1
            
        elif(item["data-image-name"] == "Chainmailarmoravailable.png" or item["data-image-name"] == "Platemailarmoravailable.png"):
            civilization['infantryarmorLine']+=1
            
        elif(item["data-image-name"] == "Faithavailable.png"):
            civilization['has_faith']+=1
            
        elif(item["data-image-name"] == "Redemptionavailable.png"):
            civilization['has_redemption']+=1
            
        elif(item["data-image-name"] == "Illuminationavailable.png"):
            civilization['has_illumination']+=1
            
        elif(item["data-image-name"] == "Atonementavailable.png"):
            civilization['has_atonement']+=1
            
        elif(item["data-image-name"] == "Blockprintingavailable.png"):
            civilization['has_blockprinting']+=1
            
        elif(item["data-image-name"] == "Herbalmedicineavailable.png"):
            civilization['has_herbalmedicine']+=1
            
        elif(item["data-image-name"] == "Theocracyavailable.png"):
            civilization['has_theocracy']+=1
            
        elif(item["data-image-name"] == "Heresyavailable.png"):
            civilization['has_heresy']+=1
            
        elif(item["data-image-name"] == "Sanctityavailable.png"):
            civilization['has_sanctity']+=1
            
        elif(item["data-image-name"] == "Fervoravailable.png"):
            civilization['has_fervor']+=1
            
        elif(item["data-image-name"] == "Masonryavailable.png"):
            civilization['has_masonry']+= 1
            
        elif(item["data-image-name"] == "Architectureavailable.png"):
            civilization['has_architecture']+= 1
            
        elif(item["data-image-name"] == "Fortifiedwallresearchavailable.png"):
            civilization['has_fortifiedwall']+= 1
            
        elif(item["data-image-name"] == "Bombardtoweravailable.png"):
            civilization['has_bombardtower']+= 1
            
        elif(item["data-image-name"] == "Siegeengineersavailable.png"):
            civilization['has_siegeengineers']+= 1
            
        elif(item["data-image-name"] == "Heatedshotavailable.png"):
            civilization['has_heatedshot']+= 1
            
        elif(item["data-image-name"] == "Arrowslitsavailable.png"):
            civilization['has_arrowslits']+= 1
            
        elif(item["data-image-name"] == "Treadmillcraneavailable.png"):
            civilization['has_treadmillcrane']+= 1
            
        elif(item["data-image-name"] == "Goldshaftminingavailable.png"):
            civilization['goldLine']+= 1
            
        elif(item["data-image-name"] == "Stoneshaftminingavailable.png"):
            civilization['stoneLine']+= 1
            
        elif(item["data-image-name"] == "Twomansawavailable.png"):
            civilization['woodLine']+= 1
            
        elif(item["data-image-name"] == "Croprotationavailable.png"):
            civilization['foodLine']+= 1
            
        elif(item["data-image-name"] == "Guildsavailable.png"):
            civilization['has_guilds']+= 1
            
        else:
            continue
    
    
    civilizations.append(civilization)
    

In [3]:
df=pandas.DataFrame(civilizations)
pandas.options.display.max_columns = None
#pandas.set_option('display.max_colwidth', 100)
#pandas.reset_option('display.max_rows')
df.set_index('civilization', inplace=True)
#df.dtypes
df=df.convert_dtypes()
df

,specialization,civilization bonuses,unique units,unique technologies,militiaLine,spearmanLine,EaglescoutLine,has_supplies,has_squires,has_arson,archerLine,skirmisherLine,handcanoneerLine,cavalryarcherLine,has_thumbring,has_parthiantactics,scoutLine,knightLine,camelriderLine,battleelephantLine,steppelancerLine,has_bloodlines,has_husbandry,batteringramLine,mangonelLine,scorpionLine,bombardcannonLine,fireshipLine,demolitionraftLine,galleyLine,cannongalleonLine,has_drydock,has_shipwright,watchtowerLine,stonewallLine,piercearmorLine,pierceatkLine,meleeatkLine,cavalryarmorLine,infantryarmorLine,has_faith,has_redemption,has_illumination,has_atonement,has_blockprinting,has_herbalmedicine,has_theocracy,has_heresy,has_sanctity,has_fervor,has_masonry,has_architecture,has_fortifiedwall,has_bombardtower,has_siegeengineers,has_heatedshot,has_arrowslits,has_treadmillcrane,goldLine,stoneLine,woodLine,foodLine,has_guilds
civilization,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Britons,a foot archer civilization.,Town Centers cost -50% wood once the Castle Ag...,Longbowman (long-ranged archer),Yeomen (+1 range for foot archers and +2 atta...,3,3,0,1,1,1,3,2,0,2,0,0,2,2,0,0,0,0,1,2,2,2,0,3,3,3,1,1,1,3,2,3,2,2,3,3,1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,2,1,2,1,1
Byzantines,a defensive civilization.,Buildings have +10%/+20%/+30%/+40% HP in the D...,Cataphract (anti-infantry heavy cavalry),Greek Fire (Fire Ships +1 range) Logistica (...,3,3,0,1,1,1,3,2,1,2,1,0,3,3,2,0,0,0,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,1,3,3,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,2,2,2,2,1
Celts,an infantry civilization.,Infantry move 15% faster. Lumberjacks work 15%...,Woad Raider (quick infantry),Stronghold (Castles and towers fire 25% faste...,3,3,0,1,0,1,2,2,0,2,0,0,3,3,0,0,0,0,1,3,3,2,0,2,3,3,1,1,1,3,2,2,1,2,2,3,1,0,0,0,0,1,0,1,1,1,1,0,1,0,1,1,1,1,2,2,1,1,1
Chinese,an archer civilization.,"Start game with three extra Villagers, but wit...",Chu Ko Nu (foot archer that fires multiple ar...,Great Wall (Walls and towers +30% HP) Rocket...,3,3,0,0,1,1,3,2,0,2,1,0,2,2,2,0,0,1,1,3,2,2,0,2,3,3,1,1,1,3,2,3,2,2,3,3,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,2,2,2,1,0
Franks,a cavalry civilization.,Farm upgrades are free. Castles are 25% cheape...,Throwing Axeman (ranged infantry),Chivalry (Stables work 40% faster) Bearded A...,3,3,0,1,1,1,2,2,1,2,0,0,2,3,0,0,0,0,1,2,2,2,1,3,3,3,1,1,0,2,2,2,1,2,3,3,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,2,1,1,2,0
Goths,an infantry civilization.,Infantry are 20%/25%/30%/35% cheaper in the Da...,"Huskarl (anti-archer infantry, almost immune ...",Anarchy (Huskarls can be created at Barracks)...,3,3,0,0,1,0,2,2,1,2,0,0,3,2,0,0,0,1,1,2,2,2,1,3,3,3,1,0,1,1,0,3,2,2,2,2,1,0,1,0,0,1,1,0,1,1,1,1,0,0,0,1,0,0,1,2,2,2,1
Japanese,an infantry civilization.,"Fishing Ships have double HP, +2 pierce armor,...",Samurai (anti-unique unit infantry),Yasama (towers fire extra arrows) Kataparuto...,3,3,0,1,1,1,3,2,1,2,1,1,2,2,0,0,0,1,1,2,2,2,0,3,2,3,2,1,1,3,2,3,2,2,2,3,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,1,2,1,2,1,0
Mongols,a cavalry archer civilization.,Cavalry archers fire 25% faster. Light Cavalry...,Mangudai (cavalry archer with an attack bonus...,Nomads (Houses retain population when destroy...,3,2,0,0,1,1,3,2,0,2,1,1,3,2,2,0,2,1,1,3,3,2,0,3,3,3,1,0,1,2,2,2,2,2,2,3,1,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,2,2,1,1,0
Persians,a cavalry civilization.,Start the game with +50 food and +50 wood. Tow...,War Elephant (slow yet very strong cavalry),Kamandaran (turns Archer gold cost into wood ...,1,3,0,1,1,1,2,2,1,2,1,1,3,3,2,0,0,1,1,3,2,2,1,3,3,3,2,1,0,2,1,3,1,2,3,3,1,0,0,0,1,1,1,0,0,1,1,1,0,0,0,1,0,0,2,2,2,2,1


In [8]:
df.to_json('Civilization_Dataset_with_bonuses.json')

In [9]:
df.drop(['specialization', 'civilization bonuses', 'unique units', 'unique technologies'], axis=1).to_json('Civilization_Dataset.json')

,militiaLine,spearmanLine,EaglescoutLine,has_supplies,has_squires,has_arson,archerLine,skirmisherLine,handcanoneerLine,cavalryarcherLine,has_thumbring,has_parthiantactics,scoutLine,knightLine,camelriderLine,battleelephantLine,steppelancerLine,has_bloodlines,has_husbandry,batteringramLine,mangonelLine,scorpionLine,bombardcannonLine,fireshipLine,demolitionraftLine,galleyLine,cannongalleonLine,has_drydock,has_shipwright,watchtowerLine,stonewallLine,piercearmorLine,pierceatkLine,meleeatkLine,cavalryarmorLine,infantryarmorLine,has_faith,has_redemption,has_illumination,has_atonement,has_blockprinting,has_herbalmedicine,has_theocracy,has_heresy,has_sanctity,has_fervor,has_masonry,has_architecture,has_fortifiedwall,has_bombardtower,has_siegeengineers,has_heatedshot,has_arrowslits,has_treadmillcrane,goldLine,stoneLine,woodLine,foodLine,has_guilds
civilization,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Britons,3,3,0,1,1,1,3,2,0,2,0,0,2,2,0,0,0,0,1,2,2,2,0,3,3,3,1,1,1,3,2,3,2,2,3,3,1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,2,1,2,1,1
Byzantines,3,3,0,1,1,1,3,2,1,2,1,0,3,3,2,0,0,0,1,3,2,1,1,3,3,3,2,1,1,3,2,3,2,1,3,3,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0,0,1,0,2,2,2,2,1
Celts,3,3,0,1,0,1,2,2,0,2,0,0,3,3,0,0,0,0,1,3,3,2,0,2,3,3,1,1,1,3,2,2,1,2,2,3,1,0,0,0,0,1,0,1,1,1,1,0,1,0,1,1,1,1,2,2,1,1,1
Chinese,3,3,0,0,1,1,3,2,0,2,1,0,2,2,2,0,0,1,1,3,2,2,0,2,3,3,1,1,1,3,2,3,2,2,3,3,1,0,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,0,2,2,2,1,0
Franks,3,3,0,1,1,1,2,2,1,2,0,0,2,3,0,0,0,0,1,2,2,2,1,3,3,3,1,1,0,2,2,2,1,2,3,3,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,2,1,1,2,0
Goths,3,3,0,0,1,0,2,2,1,2,0,0,3,2,0,0,0,1,1,2,2,2,1,3,3,3,1,0,1,1,0,3,2,2,2,2,1,0,1,0,0,1,1,0,1,1,1,1,0,0,0,1,0,0,1,2,2,2,1
Japanese,3,3,0,1,1,1,3,2,1,2,1,1,2,2,0,0,0,1,1,2,2,2,0,3,2,3,2,1,1,3,2,3,2,2,2,3,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,1,2,1,2,1,0
Mongols,3,2,0,0,1,1,3,2,0,2,1,1,3,2,2,0,2,1,1,3,3,2,0,3,3,3,1,0,1,2,2,2,2,2,2,3,1,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,2,2,1,1,0
Persians,1,3,0,1,1,1,2,2,1,2,1,1,3,3,2,0,0,1,1,3,2,2,1,3,3,3,2,1,0,2,1,3,1,2,3,3,1,0,0,0,1,1,1,0,0,1,1,1,0,0,0,1,0,0,2,2,2,2,1
